In [ ]:
#@title check GPU, number of CPU cores and threads
# %%script false --no-raise-error
%%bash
# check free ram
free -h
# check GPU (should be P100-PCIE-16GB)
nvidia-smi -L
# check CPU (should be Intel(R) Xeon(R) CPU @ 2.30 GHz)
lscpu | grep 'Model name'
# check number of cores per socket (should be Core(s) per socket: 2)
lscpu | grep 'Core(s) per socket'
# check number of threads per core (should be Thread(s) per core: 2)
lscpu | grep 'Thread(s) per core'
# utility for monitoring and managing GPU(s)
nvidia-smi
# check CUDA version in google colab
nvcc --version

In [ ]:
%%script false --no-raise-error
#@title checking gpu (python code)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
#@title check RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
#@title connect Google Drive (not used because automatically connecting to it if jupyter notebook is not shared with other accounts in Google Drive)
%%script false --no-raise-error
%cd /content/
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title link Google Drive to folder /mydrive
%%bash
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
for gdrivefolder in "/content/gdrive/My\ Drive" "/content/drive/MyDrive"; do
  if [[ -d $gdrivefolder ]]; then 
    ln -s "$gdrivefolder" /mydrive; 
  fi;
done;

#ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
#!ls -la /content/drive/MyDrive/
!ls -la /mydrive
!ls -la /mydrive/

In [ ]:
#@title check OS and kernel version
!uname -a
!lsb_release -a

In [ ]:
#@title update all packages (disabled)
%%script false --no-raise-error
%%bash
sudo apt update -y ;
sudo apt list --upgradable ;
sudo apt upgrade -y ;
sudo apt install --fix-broken --fix-missing -y ;
sudo apt autoremove;

In [ ]:
#@title build ImageMagick from source
# %%script false --no-raise-error
%%bash
# Install imagemagick for Ubuntu 18.04.5
sudo apt update -y ;
sudo apt-get install -y fontconfig fontconfig-config fonts-dejavu-core fonts-droid-fallback fonts-noto-mono ghostscript gsfonts hicolor-icon-theme libxt6 libavahi-client3 libavahi-common-data libavahi-common3 libcairo2 libcups2 libcupsfilters1 libcupsimage2 libdatrie1 libdjvulibre-text libdjvulibre21 libfftw3-double3 libfontconfig1 libgraphite2-3 libgs9 libgs9-common libharfbuzz0b libijs-0.35 libilmbase12 libjbig0 libjbig2dec0 libjpeg-turbo8 libjpeg8 liblcms2-2 liblqr-1-0  libnetpbm10 libopenexr22 libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libpaper-utils libpaper1 libpixman-1-0 libthai-data libthai0 libtiff5 libwmf0.2-7 libxcb-render0 libxcb-shm0 libxrender1 netpbm poppler-data;
sudo apt-get -qq install -y build-essential;
# wget https://www.imagemagick.org/download/ImageMagick.tar.gz
# tar xzf ImageMagick.tar.gz
# for i in ImageMagick-*; do echo cd $i; done
pwd;
if [[ -d "./ImageMagick" ]]; then rm -rf ImageMagick; fi;
git clone https://github.com/ImageMagick/ImageMagick.git ImageMagick;
cd ImageMagick;
git reset --hard $(curl --silent "https://api.github.com/repos/ImageMagick/ImageMagick/releases/latest" | grep '"target_commitish":' | sed -E 's/.*"([^"]+)".*/\1/');
./configure;
make;
sudo make install;
sudo ldconfig /usr/local/lib;
pwd;
cd ../;
pwd;

In [ ]:
#@title check compiled ImageMagick version
!magick -version

In [ ]:
#@title clone darknet repo and make some changes
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!git reset --hard ac8ebca0639f445ae456a1da05a13496ae0fcdc2
get_ipython().system("pip install pytz")
import re
from shutil import copyfile
from time import strftime
from datetime import datetime
from pytz import timezone as pytztimezone

# timestr = strftime("%Y%m%d-%H%M%S")
MSK = pytztimezone('Europe/Moscow')
datetime_msk = datetime.now(MSK)
timestr = datetime_msk.strftime("%Y%m%d-%H%M%S")
print(timestr)

cfgs = ["src/image.c", "src/image_opencv.cpp", "data/labels/make_labels.py", "src/network.c"]
# cfgs = ["src/image.c", "src/image_opencv.cpp", "data/labels/make_labels.py"]

for i in cfgs:
    input_file = open(i, 'r')
    input_read = input_file.read()
    input_file.close()
    input_file = open(i, 'w')
    if i == "src/image.c":
        input_read = re.sub(r'\s{3,4}assert\(x < m\.w && y < m\.h && c < m\.c\);', r'    // assert(x < m.w && y < m.h && c < m.c);', input_read, flags = re.M)
        input_read = re.sub(r'(float (\w*) = get_color\(\d*,\s*offset,\s*classes\)\s*;)$', r'\1 \2 = 1.0;', input_read, flags = re.M)
        input_read = re.sub(r'image label = get_label_v3\(alphabet, labelstr, \(im\.h\*\.02\)\);', r'image label = get_label_v3(alphabet, labelstr, (im.h*.03));', input_read, flags = re.M)
        input_read = re.sub(r'(strcat\(labelstr, names\[selected_detections\[i\]\.best_class\]\);\s*)$', r'\1 int k; k = selected_detections[i].best_class; char bufferj[100]; sprintf(bufferj, " : %.1f %% ", selected_detections[i].det.prob[k] * 100); strcat (labelstr, bufferj);', input_read, flags = re.M)
    elif i == "src/image_opencv.cpp":
        input_read = re.sub(r'save_mat_png\(img, "chart\.png"\);', (r'save_mat_png(img, "charts/chart-' + timestr + r'.png");'), input_read, flags = re.M)
        # input_read = re.sub(r'save_mat_png\(img, "chart\.png"\);', r'save_mat_png(img, "charts/chart.png");', input_read, flags = re.M)
    elif i == "data/labels/make_labels.py":
        input_read = re.sub(r'convert -fill black -background white -bordercolor white', r'convert -fill white -background black -bordercolor black', input_read, flags = re.M)
        input_read = re.sub(r'for (\w*) in \[(\s?\d*\s?,?){8}\]', r'for \1 in [10,20,30,40,50,60,70,80]', input_read, flags = re.M)
        input_read = re.sub(r's/\d*-1', r's/10-1', input_read, flags = re.M)
        input_read = re.sub(r"font = .*", r"font = 'Futura-Normal'", input_read, flags = re.M)
    elif i == "src/network.c":
        input_read = re.sub(r'\s{3,4}(network_predict\s*\(\s*\*net,\s*im\.data\)\s*;\s*$)', r'    if (net->batch != 1) set_batch_network(net, batch_size); \1', input_read, flags = re.M)
    input_file.write(input_read)
    input_file.close()

font = 'Futura-Normal-Regular.ttf'
copyfile(f'/mydrive/yolov4/{font}', f'/usr/local/share/fonts/{font}')

In [ ]:
#@title create labels (for bndbox)
%%bash
pwd
cd data/labels/
pwd
python3 make_labels.py
cd ../../
pwd

# ln -s /content/gdrive/My\ Drive/yolov4/charts/ charts
for gdrivefolder in "/content/gdrive/My\ Drive/yolov4/charts" "/content/drive/MyDrive/yolov4/charts"; do
  if [[ -d $gdrivefolder ]]; then 
    ln -s "$gdrivefolder" /content/darknet/charts ;
  fi;
done;


In [ ]:
#@title edit makefile for CPU or GPU (if exist)
# %cd /content/darknet
input_file = open('Makefile', 'r')
input_read = input_file.read()
input_file.close()
input_file = open('Makefile', 'w')
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
def editmakefile(param, value, input_read):
  input_read = re.sub((param + r'\s*=\s*\d*'), (param + r'=' + value), input_read, flags = re.M)
  return input_read
if gpu_info.find('failed') >= 0:
  for i in (r'AVX', r'OPENMP', r'OPENCV'):
    input_read = editmakefile(i, r'1', input_read)
else:
  for i in (r'OPENCV', r'GPU', r'CUDNN', r'CUDNN_HALF'):
    input_read = editmakefile(i, r'1', input_read)
        
input_file.write(input_read)
input_file.close()

In [ ]:
#@title build / compile Darknet from source code
%%bash
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
make

In [ ]:
#@title define helper python functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

"""
# use this to generate train.txt and test.txt
def generatetxt(imgpath):
  import os
  
  image_files = []
  os.chdir(os.path.join("data", imgpath))
  for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
      image_files.append("data/" + imgpath + "/" + filename)
  os.chdir("..")
  with open(imgpath + ".txt", "w") as outfile:
    for image in image_files:
      outfile.write(image)
      outfile.write("\n")
    outfile.close()
  os.chdir("..")
"""


gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
  
#get_ipython().system("ls -la /usr/bin/nvidia-smi")
# memory footprint support libraries/code
#get_ipython().system("ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi")

get_ipython().system("pip install psutil")
get_ipython().system("pip install humanize")
get_ipython().system("pip install gputil")

# XXX: only one GPU on Colab and isn’t guaranteed

def printm():
 import psutil
 import humanize
 import os
 import GPUtil as GPU

 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 if gpu_info.find('failed') < 0:
  GPUs = GPU.getGPUs()
  gpu = GPUs[0]
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
#@title copy dataset properties
"""
# this is where my datasets are stored within my Google Drive (I created a yolov4 folder to store all important files for custom training) 
!ls /mydrive/yolov4
"""
# %cd /content/darknet/

from shutil import copyfile

#for i in ('generate_train.py', 'generate_test.py', 'yolov4.conv.137'):
#  copyfile(f'/mydrive/yolov4/{i}', i)
for i in ('obj.data', 'obj.names', 'obj.zip'):
  copyfile(f'/mydrive/yolov4/{i}', f'data/{i}')
# copyfile('/mydrive/yolov4/yolov4.conv.137', 'yolov4.conv.137')
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4.conv.137
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
# copyfile('cfg/yolov4.cfg', 'cfg/yolov4-custom.cfg')
# copyfile('cfg/yolov4-tiny.cfg', 'cfg/yolov4-tiny-custom.cfg')

In [ ]:
#@title delete unnecessary .txt files
get_ipython().system(r"unzip data/obj.zip -d data/")

from os import remove, path
for i in ('/content/darknet/data/train/classes.txt', '/content/darknet/data/test/classes.txt'):
  if path.isfile(i):
    remove(i)

In [ ]:
#@title generate train and test .txt lists
# %cd /content/darknet/
import os
os.getcwd()
def generatetxt(selection):
  test_image_files = []
  os.chdir(os.path.join("data", selection))
  for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpeg") or filename.endswith(".jpg"):
      test_image_files.append(f"data/{selection}/" + filename)
  os.chdir("..")
  with open(f"{selection}.txt", "w") as outfile:
    for image in test_image_files:
      outfile.write(image)
      outfile.write("\n")
    outfile.close()
  os.chdir("..")

generatetxt("train")
generatetxt("test")

# Step 5: Train Your Custom Object Detector!
The time has finally come! You have made it to the moment of truth! You are now ready to train your custom YOLOv4 object detector on whatever crazy classes you have decided on. So run the following command. (-dont_show flag stops chart from popping up since Colab Notebook can't open images on the spot, -map flag overlays mean average precision on chart to see how accuracy of your model is, only add map flag if you have a validation dataset)
```
!./darknet detector train <path to obj.data> <path to custom config> yolov4.conv.137 -dont_show -map
```
**TIP:** This training could take several hours depending on how many iterations you chose in the .cfg file. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit **Enter**
```
function ClickConnect(){
console.log("Working"); 
console.log(new Date().getHours() + ":" + new Date().getMinutes() + ":" + new Date().getSeconds())
// console.log(new Date());
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```
or right clock on connect button and paste this to browser console:
```
function ClickConnect() {
  console.log("Clicked on connect button");
  document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)
```
Looks like this, it will click the screen every 10 minutes so that you don't get kicked off for being idle! HACKS!

Use the code below to automatically reconnect if conntection lost, and automatically start train (run all cells, pressing F9):
```
function ClickConnect(){
console.log("Working. Time: " + new Date().getHours() + ":" + new Date().getMinutes() + ":" + new Date().getSeconds()); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click()
}
function runAll() {
  const F9Event = {key: "F9", code: "F9", metaKey: true, keyCode: 120};
  document.dispatchEvent(new KeyboardEvent("keydown", F9Event));
}
function DismissDialog() {
    if (document.querySelector("colab-dialog > paper-dialog > div.buttons > paper-button#cancel") != null) {
        document.querySelector("colab-dialog > paper-dialog > div.buttons > paper-button#cancel").click();
        return console.log(setTimeout(CheckConnected, 1800000));
    } else {
        runAll();
        return console.log(setTimeout(CheckConnected, 60000));
    };
}
function CheckConnected() {
    if (colab.global.notebook.kernel.isConnected() == false) { 
        colab.global.notebook.connectToKernel();
        return console.log(setTimeout(DismissDialog, 30000));
    } else {
        ClickConnect();
        return console.log(setTimeout(CheckConnected, 60000));
    }
}
console.log(CheckConnected());
```


In [ ]:
#@title stop 'Run all' (disabled)
%%script false --no-raise-error
# This stops 'Run all' at this cell by causing an error
assert False

In [ ]:
#@title edit cfgs for training
import re
from shutil import copyfile

# origincfgs = ["cfg/yolov4.cfg", "cfg/yolov4-tiny.cfg"]
cfgs = ["cfg/yolov4-custom.cfg", "cfg/yolov4-tiny-custom.cfg"]
# for i in range(len(cfgs)):
    # copyfile(origincfgs[i], cfgs[i])

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)

for i in cfgs:
    input_file = open(i, 'r')
    input_read = input_file.read()
    input_file.close()
    input_file = open(i, 'w')
    # set batch size to 64
    input_read = re.sub(r'batch\s*=\s*\d*', r'batch=64', input_read, flags = re.M)
    # set subdivisions to 32 for yolov4 and 4 for yolov4-tiny
    if i == "cfg/yolov4-custom.cfg":
        if gpu_info.find('failed') < 0:
            input_read = re.sub(r'subdivisions\s*=\s*\d*', r'subdivisions=32', input_read, flags = re.M)
        else:
            input_read = re.sub(r'subdivisions\s*=\s*\d*', r'subdivisions=64', input_read, flags = re.M)
    elif i == "cfg/yolov4-tiny-custom.cfg":
        if gpu_info.find('failed') < 0:
            input_read = re.sub(r'subdivisions\s*=\s*\d*', r'subdivisions=4', input_read, flags = re.M)
        else:
            input_read = re.sub(r'subdivisions\s*=\s*\d*', r'subdivisions=8', input_read, flags = re.M)
    # set channels number to 1 (grayscale, not RGB)
    input_read = re.sub(r'channels\s*=\s*\d*', r'channels=1', input_read, flags = re.M)
    # set hue to 0 (do not randomize)
    # input_read = re.sub(r'hue\s*=.*', r'hue=0 # set to 0.1 to randomly change hue during training', input_read, flags = re.M)
    # set number of filters before yolo layer to 54 ((classes+5)*3)
    # input_read = re.sub(r'(size=1\nstride=1\npad=1\n)filters\s*=\s*\d*', r'\1filters=54', input_read, flags = re.M)
    input_read = re.sub(r'(size=1\nstride=1\npad=1\n)filters\s*=\s*\d*', r'\1filters=45', input_read, flags = re.M)
    # set number of yolo classes to 13
    # input_read = re.sub(r'classes\s*=\s*\d*', r'classes = 13', input_read, flags = re.M)
    input_read = re.sub(r'classes\s*=\s*\d*', r'classes = 10', input_read, flags = re.M)
    # set max_batches to 26000 (13 classes * 2000)
    # input_read = re.sub(r'max_batches\s*=\s*\d*', r'max_batches = 26000', input_read, flags = re.M)
    input_read = re.sub(r'max_batches\s*=\s*\d*', r'max_batches = 100000', input_read, flags = re.M)
    # set steps to 20800 (26000*0,8) and 23400 (26000*0,9)
    # input_read = re.sub(r'steps\s*=\s*\d*\s*,\s*\d*', r'steps = 20800, 23400', input_read, flags = re.M)
    input_read = re.sub(r'steps\s*=\s*\d*\s*,\s*\d*', r'steps = 80000, 90000', input_read, flags = re.M)
    #set max_chart_lost (maximum Y on chart.png) to 5
    if re.search(r'max_chart_loss', input_read):
        input_read = re.sub(r'max_chart_loss\s*=\s*\d*\s*', r'max_chart_loss = 1', input_read, flags = re.M)
    else:
        input_read = re.sub(r'(hue.*)', r'\1\nmax_chart_loss=1', input_read, flags = re.M)
    input_file.write(input_read)
    input_file.close()

In [ ]:
#@title start/resume train
# %%script false --no-raise-error
%%bash
# model="yolov4-custom"
model="yolov4-tiny-custom"
# pretrainedweights="yolov4.conv.137"
pretrainedweights="yolov4-tiny.conv.29"
outputs="/mydrive/yolov4/outputs"
weightsdir="/mydrive/yolov4/backup"
#get_ipython().system(r"./darknet detector train data/obj.data cfg/$model.cfg $weightsdir/$model\_last.weights -dont_show -map >> $outputs/$(date +%Y.%m.%d)-$model.txt 2>&1")
if [ -f $outputs/$(date +%Y.%m.%d)-$model-RAM.txt ]; then 
  echo -e "\n\n\n\n\nRAMtotal RAMused RAMfree" >> $outputs/$(date +%Y.%m.%d)-$model-RAM.txt; 
else 
  echo "RAMtotal RAMused RAMfree" >> $outputs/$(date +%Y.%m.%d)-$model-RAM.txt; 
fi;
if [[ $(nvidia-smi) != *"failed"* ]]; then
  if [[ ! -f $outputs/$(date +%Y.%m.%d)-$model-VRAM.csv ]]; then 
    echo "VRAMutilization, VRAMtotal, VRAMused, VRAMfree" >> $outputs/$(date +%Y.%m.%d)-$model-VRAM.csv ;
  fi;
fi;
# resume start new train or resume previous if weights file persist
if [[ -f $weightsdir/$model\_last.weights ]]; then
    ./darknet detector train data/obj.data cfg/$model.cfg $weightsdir/$model\_last.weights -dont_show -map >> $outputs/$(date +%Y.%m.%d)-$model.txt 2>&1 & 
elif [[ ! -f $weightsdir/$model\_final.weights ]]; then
    ./darknet detector train data/obj.data cfg/$model.cfg $pretrainedweights -dont_show -map >> $outputs/$(date +%Y.%m.%d)-$model.txt 2>&1 & 
fi;
sleep 60;
if [[ $(nvidia-smi) != *"failed"* ]]; then
  while [ $(pgrep darknet | wc -c) != 0 ]; do free -m | grep "Mem:" | awk '{print $2,$3,$4}' >> $outputs/$(date +%Y.%m.%d)-$model-RAM.txt; sleep 60; done & 
  while [ $(pgrep darknet | wc -c) != 0 ]; do nvidia-smi --query-gpu=utilization.memory,memory.total,memory.used,memory.free --format=csv | tail -n 1 >> $outputs/$(date +%Y.%m.%d)-$model-VRAM.csv; sleep 60; done;
else 
  while [ $(pgrep darknet | wc -c) != 0 ]; do free -m | grep "Mem:" | awk '{print $2,$3,$4}' >> $outputs/$(date +%Y.%m.%d)-$model-RAM.txt; sleep 60; done;
fi;

In [ ]:
#@title showing chart.png
# %%script false --no-raise-error

# show chart.png of how custom object detector did with training
imShow((r'charts/chart-' + timestr + r'.png'))
download((r'charts/chart-' + timestr + r'.png'))

In [ ]:
#@title measure average precision
# %%script false --no-raise-error
%%bash
# model="yolov4-custom"
model="yolov4-tiny-custom"
outputs="/mydrive/yolov4/outputs"
weightsdir="/mydrive/yolov4/backup"
./darknet detector map data/obj.data cfg/$model.cfg $weightsdir/$model\_best.weights -dont_show -map 2>&1

In [ ]:
#@title edit cfgs for testing trained model
%%script false --no-raise-error
import re
from shutil import copyfile

# copy yolov4 to yolov4-custom and yolov4-tiny to yolov4-tiny-custom
# origincfgs = ["cfg/yolov4.cfg", "cfg/yolov4-tiny.cfg"]
cfgs = ["cfg/yolov4-custom.cfg", "cfg/yolov4-tiny-custom.cfg"]
# for i in range(2):
    # copyfile(origincfgs[i], cfgs[i])

for i in cfgs:
    input_file = open(i, 'r')
    input_read = input_file.read()
    input_file.close()
    input_file = open(i, 'w')
    # set batch size to 1
    input_read = re.sub(r'batch\s*=\s*\d*', r'batch=1', input_read, flags = re.M)
    # set subdivisions to 1
    input_read = re.sub(r'subdivisions\s*=\s*\d*', r'subdivisions=1', input_read, flags = re.M)
    # set channels number to 1 (grayscale, not RGB)
    input_read = re.sub(r'channels\s*=\s*\d*', r'channels=1', input_read, flags = re.M)
    # set hue to 0 (do not randomize)
    # input_read = re.sub(r'hue\s*=.*', r'hue=0 # set to 0.1 to randomly change hue during training', input_read, flags = re.M)
    # set number of filters before yolo layer to 54 ((classes+5)*3)
    # input_read = re.sub(r'(size=1\nstride=1\npad=1\n)filters\s*=\s*\d*', r'\1filters=54', input_read, flags = re.M)
    input_read = re.sub(r'(size=1\nstride=1\npad=1\n)filters\s*=\s*\d*', r'\1filters=45', input_read, flags = re.M)
    # set number of yolo classes to 13
    # input_read = re.sub(r'classes\s*=\s*\d*', r'classes = 13', input_read, flags = re.M)
    input_read = re.sub(r'classes\s*=\s*\d*', r'classes = 10', input_read, flags = re.M)
    # set max_batches to 26000 (13 classes * 2000)
    # input_read = re.sub(r'max_batches\s*=\s*\d*', r'max_batches = 26000', input_read, flags = re.M)
    input_read = re.sub(r'max_batches\s*=\s*\d*', r'max_batches = 100000', input_read, flags = re.M)
    # set steps to 20800 (26000*0,8) and 23400 (26000*0,9)
    # input_read = re.sub(r'steps\s*=\s*\d*\s*,\s*\d*', r'steps = 20800, 23400', input_read, flags = re.M)
    input_read = re.sub(r'steps\s*=\s*\d*\s*,\s*\d*', r'steps = 80000, 90000', input_read, flags = re.M)
    #set max_chart_lost (maximum Y on chart.png) to 5
    if re.search(r'max_chart_loss', input_read):
        input_read = re.sub(r'max_chart_loss\s*=\s*\d*\s*', r'max_chart_loss = 1', input_read, flags = re.M)
    else:
        input_read = re.sub(r'(hue.*)', r'\1\nmax_chart_loss=1', input_read, flags = re.M)
    input_file.write(input_read)
    input_file.close()

In [ ]:
%%script false --no-raise-error
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
#!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/car2.jpg -thresh 0.3
#imShow('predictions.jpg')
#get_ipython().system(r"randomimage=$(ls data/test/*.jpeg | sort -R | tail -1); ./darknet detector test data/obj.data cfg/yolov4-tiny-custom.cfg /mydrive/yolov4/backup/yolov4-tiny-custom_last.weights $randomimage -thresh 0.3; echo $randomimage")
for fname in os.listdir('./data/test/'):
    if fname.endswith('jpg'):
      extension="jpg"
    elif fnmame.endswith('jpeg'):
      extension="jpeg"

!randomimage=$(ls data/test/*.$extension | sort -R | tail -1); ./darknet detector test data/obj.data cfg/yolov4-tiny-custom.cfg /mydrive/yolov4/backup/yolov4-tiny-custom_last.weights $randomimage -thresh 0.3; echo $randomimage
imShow('predictions.jpg')
download('predictions.jpg')

In [ ]:
%%script false --no-raise-error
#use this to check amount of free VRAM (GPU RAM)
printm()